In [ ]:
# 导入必要的库
import numpy as np            # 导入numpy库，用于进行高效的数值计算，特别是处理数组和矩阵。
from PIL import Image         # 从PIL (Pillow) 库导入Image模块，用于图像处理，如打开、调整大小和格式转换。
import onnxruntime as ort     # 导入onnxruntime库并别名为ort，它是运行ONNX模型的引擎，可以高效地执行深度学习模型的推理。


# 定义预处理函数，用于将图片转换为模型所需的输入格式
def preprocess(image_path):
    # `preprocess` 函数接收图片路径作为输入，并将图片转换为模型期望的格式。
    input_shape = (1, 1, 64, 64)    # 模型输入期望的形状，这里是 (N, C, H, W)，N=batch size, C=channels, H=height, W=width
    # 定义模型输入的预期形状：
    # `N=1` 表示批次大小为1，即每次处理一张图片。
    # `C=1` 表示颜色通道数为1，即灰度图。
    # `H=64, W=64` 表示图像的高度和宽度都是64像素。

    img = Image.open(image_path).convert('L')    # 打开图像文件并将其转换为灰度图  1分
    # 使用PIL的`Image.open()`函数打开指定路径的图片文件。
    # `.convert('L')` 将图片转换为Luminance（亮度）模式，即单通道的灰度图。
    # 情感识别模型通常期望灰度图像，以减少计算复杂性并专注于面部特征。

    img = img.resize((64, 64), Image.ANTIALIAS)    # 调整图像大小到模型输入所需的尺寸
    # 将图像调整为64x64像素。
    # `Image.ANTIALIAS` 是一种高质量的图像重采样滤波器，用于在缩放时减少锯齿和失真。
    # 这确保了图像尺寸与模型的输入要求一致。

    img_data = np.array(img, dtype=np.float32)    # 将PIL图像对象转换为numpy数组，并确保数据类型是float32
    # 将处理后的PIL图像对象 `img` 转换为NumPy数组。
    # `dtype=np.float32` 将数组元素的数据类型指定为32位浮点数，这是深度学习模型通常要求的输入类型，有助于精度和兼容性。

    # 调整数组的形状以匹配模型输入的形状
    img_data = np.expand_dims(img_data, axis=0)  # 添加 batch 维度
    # 使用 `np.expand_dims()` 在数组的最前面（轴0）添加一个新维度。
    # 原始 `img_data` 形状是 (64, 64)。添加批次维度后变为 (1, 64, 64)。
    # `1` 表示批次大小为1，因为模型通常期望输入是一个批次（Batch）的图像。

    img_data = np.expand_dims(img_data, axis=1)  # 添加 channel 维度
    # 再次使用 `np.expand_dims()` 在数组的第二个位置（轴1）添加一个新维度。
    # 上一步形状是 (1, 64, 64)。添加通道维度后变为 (1, 1, 64, 64)。
    # `1` 在这里表示图像有一个颜色通道，即它是灰度图。
    # 这种 `(batch_size, channels, height, width)` 的顺序是许多深度学习框架（如PyTorch）和ONNX模型所期望的。

    assert img_data.shape == input_shape, f"Expected shape {input_shape}, but got {img_data.shape}"    # 确保最终的形状与模型输入要求的形状一致
    # 使用 `assert` 语句进行断言检查。如果 `img_data.shape` 不等于 `input_shape`，则会抛出 `AssertionError`，并附带错误信息。
    # 这是一种重要的调试和验证机制，确保图像预处理的输出符合模型的期望。

    return img_data    # 返回预处理后的图像数据
    # 函数返回最终处理好的NumPy数组，它现在是模型可以直接接受的格式。


# 定义情感类别与数字标签的映射表 3分
emotion_table = {'neutral':0, 'happiness':1, 'surprise':2,'sadness':3, 'anger':4, 'disgust':5, 'fear':6, 'contempt':7}
# 创建一个Python字典 `emotion_table`，用于将情感的英文名称（字符串）映射到对应的整数标签。
# 这在模型输出数字标签后，可以方便地将其转换回可读的情感名称。
# 例如，如果模型预测标签为 0，则对应 'neutral' 情感。


# 加载模型 3分
ort_session = ort.InferenceSession('emotion-ferplus.onnx')    # 使用onnxruntime创建一个会话，用于加载并运行模型
# 使用 `ort.InferenceSession()` 加载名为 'emotion-ferplus.onnx' 的预训练ONNX模型。
# 这个模型是基于FER2013数据集或其变体（如FER+）训练的，专门用于识别面部表情所表达的情感。
# `ort_session` 对象将用于与加载的模型进行交互以进行推理。

# 加载本地图片并进行预处理 3分
input_data = preprocess('img_test.png')
# 调用之前定义的 `preprocess` 函数，将名为 'img_test.png' 的图片文件加载并进行预处理。
# 预处理后的数据 `input_data` 将是形状正确的NumPy数组。


# 准备输入数据，确保其符合模型输入的要求
ort_inputs = {ort_session.get_inputs()[0].name: input_data}    # ort_session.get_inputs()[0].name 是获取模型的第一个输入的名字
# 创建一个字典 `ort_inputs`，它包含了模型进行推理所需的所有输入数据。
# 字典的键是模型第一个输入层（节点）的名称，通过 `ort_session.get_inputs()[0].name` 动态获取。
# 字典的值是经过预处理的图像数据 `input_data`。

# 运行模型，进行预测 3分
ort_outs = ort_session.run(None, ort_inputs)
# 使用 `ort_session.run()` 方法执行模型的推理（即情感识别）。
# 第一个参数 `None` 表示我们希望获取模型所有的输出结果。
# 第二个参数 `ort_inputs` 是前面准备好的输入字典。
# `ort_outs` 将是一个列表，包含模型的所有输出（通常是未经Softmax处理的logits）。

# 解码模型输出，找到预测概率最高的情感类别 3分
predicted_label = np.argmax(ort_outs[0])
# `ort_outs[0]` 通常是模型输出列表中的第一个元素，这是一个包含各个情感类别得分或概率的NumPy数组。
# `np.argmax()` 函数会返回这个数组中最大值所对应的索引。
# 这个索引 `predicted_label` 就是模型认为该图像表达的情感对应的数字标签。

# 根据预测的标签找到对应的情感名称 3分
predicted_emotion = list(emotion_table.keys())[predicted_label]
# `emotion_table.keys()` 获取 `emotion_table` 字典中所有的键（即情感名称）。
# `list(...)` 将这些键转换为一个列表。
# 然后，使用 `predicted_label` 作为索引，从这个列表中找出对应的情感名称。
# 这样，数字标签就被转换回了人类可读的情感描述。

# 输出预测的情感
print(f"Predicted emotion: {predicted_emotion}")
# 打印最终的预测结果，即模型识别出的图像所表达的情感。
